In [5]:
# activate environment 
using Pkg
Pkg.activate(@__DIR__);
Pkg.instantiate();

  Activating environment at `~/Desktop/ForwardDiff_examples/Project.toml`


In [6]:
using LinearAlgebra, ForwardDiff

## Basic usage

In [48]:
function foo1(x)
    # vector input, scalar output
    return sin(x[1]) + cos(x[2])
end
function foo2(x)
    # vector input, vector output
    return [sin(x[1])*x[2];cos(x[2])*x[1]]
end

# calculate a gradient with forward diff 

let # we just use this to avoid creating global variables
    
    # evaluate the gradient and hessian of foo1 at x1 
    x1 = 5*randn(2);
    @show ∇foo1 = ForwardDiff.gradient(foo1, x1);
    @show ∇²foo1 = ForwardDiff.hessian(foo1, x1);
    
    # evluate the jacobian of foo2 at x1 
    @show ∂foo1_∂x = ForwardDiff.jacobian(foo2,x1);
    
end

∇foo1 = ForwardDiff.gradient(foo1, x1) = [-0.898379851108036, 0.41843306797905394]
∇²foo1 = ForwardDiff.hessian(foo1, x1) = [-0.43921935649866695 0.0; 0.0 -0.9082476356267801]
∂foo1_∂x = ForwardDiff.jacobian(foo2, x1) = [0.3878480170333289 0.43921935649866695; 0.9082476356267801 1.1242723747224044]


2×2 Matrix{Float64}:
 0.387848  0.439219
 0.908248  1.12427

## Derivatives for functions with multiple input arguments

In [35]:
# now let's calculate derivatives for functions with multiple inputs 
function dynamics(x,a,b,c)
    return [x[1]*a; b*c*x[2]*x[1]]
end

let 
    x1 = randn(2)
    a = randn()
    b = randn()
    c = randn()
    
    # evaluate the jacobian with respect to x, given a, b, and c
    A = ForwardDiff.jacobian(_x -> dynamics(_x, a, b, c), x1)
end

2×2 Matrix{Float64}:
 0.137575   0.0
 0.134799  -0.453855

## Derivatives of composite functions

In [45]:
function f(x)
    return x[1]*x[2]
end
function g(x)
    return [x[1]^2; x[2]^3]
end

let 
    x1 = 2*randn(2)
    
    # using gradient of the composite function
    ∇f_1 = ForwardDiff.gradient(_x -> f(g(_x)), x1)
    
    # using the chain rule 
    J = ForwardDiff.jacobian(g, x1)
    ∇f_2 = J'*ForwardDiff.gradient(f, g(x1))
    
    @show norm(∇f_1 - ∇f_2)
end

norm(∇f_1 - ∇f_2) = 0.0


0.0

## Creating zero arrays inside a function
First we detail the incorrect way to do this

In [46]:
function f_zero_1(x)
    
    # diffing through this function will error on this line 
    xdot = zeros(length(x))
    
    xdot[1] = x[1]*x[2]
    xdot[2] = x[2]^2
    
    return xdot 
end

let 
    # try and calculate the jacobian of f_zero_1 on x1
    x1 = randn(2)
    ForwardDiff.jacobian(f_zero_1,x1) # this will fail! 
end

LoadError: MethodError: no method matching Float64(::ForwardDiff.Dual{ForwardDiff.Tag{typeof(f_zero_1), Float64}, Float64, 2})
[0mClosest candidates are:
[0m  (::Type{T})(::Real, [91m::RoundingMode[39m) where T<:AbstractFloat at rounding.jl:200
[0m  (::Type{T})(::T) where T<:Number at boot.jl:760
[0m  (::Type{T})([91m::AbstractChar[39m) where T<:Union{AbstractChar, Number} at char.jl:50
[0m  ...

Now we create the zero array correctly

In [47]:
function f_zero_2(x)
    
    # by using `eltype(x)`, we can now diff through this 
    xdot = zeros(eltype(x), length(x))
    
    xdot[1] = x[1]*x[2]
    xdot[2] = x[2]^2
    
    return xdot 
end

let 
    # calculate the jacobian of f_zero_2 on x1
    x1 = randn(2)
    ForwardDiff.jacobian(f_zero_2,x1) # this will work!
end

2×2 Matrix{Float64}:
 -0.643988  -0.61973
 -0.0       -1.28798